In [ ]:
# @title Import Libraries & Required Files
!git clone https://github.com/saiflab/ATMeQ.git
# Import numpy
import numpy as np
# Import pandas
import pandas as pd
# Import seaborn
import seaborn as sns
# Import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from sklearn.preprocessing import StandardScaler


In [2]:
# @title Load ATMeQ ML Model
# Load the ATMeQ model using pickle
with open('/content/ATMeQ/ATMeQ.pkl', 'rb') as f:
    ATMeQ_model = pickle.load(f)


In [ ]:
# @title Upload Input (RNA-Seq) File
# Step 1: Import the necessary library for file upload
from google.colab import files

# Step 2: Prompt the user to upload a CSV file from their local computer
# This will open a file selection dialog to choose the file to upload.
uploaded = files.upload()

# Step 3: Notify the user that the upload process is complete
# The 'uploaded' dictionary will contain the uploaded file(s) as key-value pairs (filename: file content).
print("Upload complete! You can now use the uploaded file(s).")


In [ ]:
# @title Pre-processing Data
# Step-by-step process to read and load a CSV file into a pandas DataFrame

# Step 1: Display the names of the uploaded files
# This will print the names of any files that have been uploaded.
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')

# Step 2: Check if any files were uploaded
if uploaded:
    # Step 3: Get the name of the first uploaded file
    file_name = list(uploaded.keys())[0]

    # Step 4: Load the file into a pandas DataFrame
    # The 'header=0' specifies that the first row contains column names.
    # The 'index_col=0' means the first column will be used as the index.
    df = pd.read_csv(file_name, header=0, index_col=0)

    # Confirm that the file has been successfully loaded
    print(f'The file "{file_name}" has been successfully loaded into the DataFrame called "df".')
else:
    # Step 5: Let the user know no files were uploaded
    print('No CSV file was uploaded. Please upload a file to continue.')

# List of selected gene columns
selected_columns = [
    'UTY', 'MIR12120', 'HTR6', 'NPY5R', 'TFAP2B', 'CHRNA6', 'BALR6', 'GPR50', 'HAND2', 'CD70', 'ESM1', 'ITGB4', 'OTOF', 'ZIM2', 'ASNSP1', 'CBLN4', 'PEG3-AS1', 'WNT3A', 'PAX3', 'NR4A2'
]
# Select the columns that are present in the DataFrame
df_selected = df[selected_columns] if all(col in df.columns for col in selected_columns) else None

# Check if the selection was successful
if df_selected is not None:
    print("Selected columns have been successfully extracted.")
else:
    print("Some columns were not found in the DataFrame.")

# Step 1: Assign the selected DataFrame to the variable 'X'
# Assuming 'df_selected' is already defined and contains the selected data from the original DataFrame.
X = df_selected

# Step 2: Ensure 'X' is explicitly converted into a pandas DataFrame
# This is useful if 'df_selected' might not already be a DataFrame (e.g., it could be a Series or other data structure).
X = pd.DataFrame(X)

# Step 3: Display the DataFrame to confirm its contents
X.head()


In [8]:
# @title Scaling Data
# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the data and transform it
X_scaled = scaler.fit_transform(X)

# Now, X_scaled contains the scaled data and can be used for predictions


In [ ]:
# @title Make Predictions
#Import the ATMeQ model
ATMeQ = ATMeQ_model
#Run ATMeQ
ATMeQ_predictions = ATMeQ.predict(X_scaled)
#Convert the ATMeQ_predictions to a dataframe
ATMeQ_predictions = pd.DataFrame(ATMeQ_predictions, columns=['Predictions'])
#Map the predictions: 0 as Non-AD and 1 as AD
ATMeQ_predictions['Predictions'] = ATMeQ_predictions['Predictions'].map({0: 'Non-ALS', 1: 'ALS'})
#Add the sample column next to predictions
ATMeQ_predictions['Sample'] = X.index
predictions = ATMeQ.predict(X)

In [ ]:
# @title View Results
ATMeQ_predictions

In [ ]:
# @title Save the Results
# Specify the file path where you want to save the .csv file in Colab
csv_file_path = '/content/ATMeQ_predictions.csv'

# Save the DataFrame as a .csv file
ATMeQ_predictions.to_csv(csv_file_path, index=False)

# Download the .csv file to your local machine
from google.colab import files
files.download(csv_file_path)